# Data extraction
parameters:
- run date: 2023_02_22
- dataset: 2023_02_01
- fetch_api = "http://candy.hesge.ch:8900/SIBiLS/fetch"

In [10]:

import os
from base_fns import get_local_folder
project_path = os.path.join(get_local_folder(), '..' )
%cd $project_path

from src.utils import split_list

from pathlib import Path
from IPython.display import display
from tqdm import tqdm
import requests
import orjson
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles


/home/paulvanrijen/data/CellTriage



## PMIDS




In [20]:
# read samples
NEG_FP = "./data/20230201/publication_negative_set.tsv"
POS_FP = "./data/20230201/publication_positive_set.tsv"

with open(NEG_FP, "r") as f:
    neg_set = f.read().splitlines() [1:]
    # neg_set = neg_set[1:]
print(f"#neg pmids: {len(neg_set)}")

with open(POS_FP, "r") as f:
    pos_set = f.read().splitlines() [1:]
print(f"#pos pmids: {len(pos_set)}")


#neg pmids: 644
#pos pmids: 22719


## SIBiLS

### negatives

In [22]:

URL_API = "http://candy.hesge.ch:8900/SIBiLS/fetch"
NEG_JSON_FP = "./data/20230201/SIBiLS_v3_run2/publication_negative_set.json"
CHUNK_SIZE = 500

concat_response = []
for chunk in tqdm(split_list(neg_set, CHUNK_SIZE)):
    params = {"col": "medline",
              "ids": ",".join(chunk)}
    r = requests.post(url=URL_API, params=params)
    concat_response.extend(orjson.loads(r.text))

with open(NEG_JSON_FP, "wb") as f:
    f.write(orjson.dumps(concat_response))


print("#neg documents fetched:", len(concat_response))

2it [00:12,  6.28s/it]


#neg documents fetched: 583


In [25]:
NEG_FAILED_JSON = "./data/20230201/SIBiLS_v3_run2/publication_negative.failed.json"

with open(NEG_JSON_FP, "rb") as f:
    out_neg_json = orjson.loads(f.read())
    out_neg_pmids = set([doc["record"]["pmid"] for doc in out_neg_json])
in_neg_pmids = set(neg_set)
delta_neg_pmids = in_neg_pmids - out_neg_pmids

with open(NEG_FAILED_JSON, "wb") as f:
    f.write(orjson.dumps(list(delta_neg_pmids)))


print("#neg documents not fetched:", len(delta_neg_pmids))
print(delta_neg_pmids)


#neg documents not fetched: 61
{'34941232', '34941234', '34941230', '34941231', '20301685', '35274911', '35050561', '31643973', '34941236', '36282940', '34648249', '34003615', '35274906', '26389331', '35267247', '24175354', '35050562', '34648250', '36351164', '28520346', '34941233', '36351161', '34097371', '20301750', '36351157', '34941235', '20945554', '36512654', '20301483', '35274912', '20301490', '29939552', '34756845', '20301652', '36351156', '20301779', '36351159', '34756847', '34914343', '31643906', '35704653', '35274913', '24027799', '34648251', '34941228', '26389437', '31082146', '20301761', '27336128', '35032962', '20301508', '34756846', '35420740', '36351163', '33270410', '20301336', '26389163', '36351160', '36351158', '34941229', '36351162'}


Some examples:

http://candy.hesge.ch:8900/SIBiLS/fetch?col=medline&ids=36351161,35704653,26389163
https://pubmed.ncbi.nlm.nih.gov/36351161/  
https://pubmed.ncbi.nlm.nih.gov/35704653/  
https://pubmed.ncbi.nlm.nih.gov/26389163/  
